In [181]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D ,MaxPool2D, Flatten ,Dense,Dropout
from keras import optimizers
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.callbacks import ReduceLROnPlateau

In [161]:
data1=pd.read_csv('./datasets/train.csv')
data2=pd.read_csv('./datasets/test.csv')

In [162]:
data1.head()

label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0      1       0       0       0       0       0       0       0       0   
1      0       0       0       0       0       0       0       0       0   
2      1       0       0       0       0       0       0       0       0   
3      4       0       0       0       0       0       0       0       0   
4      0       0       0       0       0       0       0       0       0   

   pixel8    ...     pixel774  pixel775  pixel776  pixel777  pixel778  \
0       0    ...            0         0         0         0         0   
1       0    ...            0         0         0         0         0   
2       0    ...            0         0         0         0         0   
3       0    ...            0         0         0         0         0   
4       0    ...            0         0         0         0         0   

   pixel779  pixel780  pixel781  pixel782  pixel783  
0         0         0         0         0         0  
1         0         0         0         0         0  
2         0         0         0         0         0  
3         0         0         0         0         0  
4         0         0         0         0         0  

[5 rows x 785 columns]

In [163]:
x=data1.iloc[:,1:].values.reshape(len(data1),28,28,1)
y=data1.iloc[:,0].values
x=np.array(x)
x.shape

(42000, 28, 28, 1)

In [164]:
x=x/255.0

In [165]:
df_y=to_categorical(y,num_classes=10)

In [166]:
df_y=np.array(df_y)

In [167]:
df_y

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [168]:
df_y.shape

(42000, 10)

In [194]:
seed=2
np.random.seed(seed)
x_train,x_val,y_train,y_val=train_test_split(x,df_y,test_size=0.1,random_state=seed)
print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

(37800, 28, 28, 1) (37800, 10) (4200, 28, 28, 1) (4200, 10)


In [195]:
#model
model=Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(28,28,1)))
model.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu',))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=(5,5), padding='same', activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(5,5), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(120, activation='relu'))
model.add(Dense(84, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_36 (Conv2D)           (None, 28, 28, 32)        832       
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 28, 28, 32)        25632     
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_23 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 14, 14, 64)        51264     
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 14, 14, 64)        102464    
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 7, 7, 64)          0         
__________

In [182]:
lr_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                 patience=3, 
                                 verbose=1, 
                                 factor=0.5, 
                                 min_lr=0.00001)

In [196]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [197]:
model.fit(x_train,y_train,epochs=5,batch_size=100,validation_data=(x_val,y_val),callbacks=[lr_reduction])

Train on 37800 samples, validate on 4200 samples
Epoch 1/5
37800/37800 [==============================] - 474s 13ms/step - loss: 0.2245 - acc: 0.9279 - val_loss: 0.0600 - val_acc: 0.9821
Epoch 2/5
37800/37800 [==============================] - 512s 14ms/step - loss: 0.0557 - acc: 0.9832 - val_loss: 0.0431 - val_acc: 0.9869
Epoch 3/5
37800/37800 [==============================] - 505s 13ms/step - loss: 0.0411 - acc: 0.9873 - val_loss: 0.0558 - val_acc: 0.9836
Epoch 4/5
37800/37800 [==============================] - 497s 13ms/step - loss: 0.0324 - acc: 0.9899 - val_loss: 0.0418 - val_acc: 0.9864
Epoch 5/5
37800/37800 [==============================] - 510s 13ms/step - loss: 0.0278 - acc: 0.9910 - val_loss: 0.0375 - val_acc: 0.9910


In [198]:
test=data2.iloc[:,0:].values.reshape(len(data2),28,28,1)
test.shape

(28000, 28, 28, 1)

In [199]:
test=test/255.0

In [200]:
predictions=model.predict_classes(test)

In [201]:
submissions = pd.DataFrame({"ImageId": list(range(1,len(predictions)+1)),
                            "Label": predictions})
submissions.head()

ImageId  Label
0        1      2
1        2      0
2        3      9
3        4      9
4        5      3

In [202]:
submissions.to_csv('submit.csv',index=False)